In [ ]:
#lstm
import warnings
import yfinance as yf
import pandas as pd
import os
from flask import Flask, render_template, request, jsonify
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from datetime import datetime, timedelta
from flask_cors import CORS  # Ensure you have flask_cors installed


warnings.simplefilter(action='ignore', category=FutureWarning)

app = Flask(__name__)
CORS(app)  # Enable CORS

# Function to fetch historical data for a specific index
def fetch_index_data(ticker):
    index = yf.Ticker(ticker)
    hist = index.history(period="1y")
    return hist

indexes = {
    "Nifty 50": "^NSEI",
    "Sensex": "^BSESN",
    # Add more indexes as needed
}

index_data = {name: fetch_index_data(ticker) for name, ticker in indexes.items()}

# Function to fetch data for global market indexes
def fetch_global_markets():
    global_indexes = {
        "^DJI": "Dow Jones",
        "^IXIC": "NASDAQ",
        "^GSPC": "S&P 500",
        "^FTSE": "FTSE 100",
        "^N225": "Nikkei 225",
        "^HSI": "Hang Seng Index",
        "^SSEC": "Shanghai Composite",
        "^GDAXI": "DAX",
        "^FCHI": "CAC 40",
        "^STOXX50E": "Euro Stoxx 50"
    }
    data = []
    for ticker, name in global_indexes.items():
        index = yf.Ticker(ticker)
        info = index.history(period="1d")  # Get the latest data
        if not info.empty:
            info = info.iloc[-1]
            data.append({
                "Symbol": name,
                "Price": info["Close"],
                "Change": info["Close"] - info["Open"],
                "ChangePct": ((info["Close"] - info["Open"]) / info["Open"]) * 100
            })
    return data

# Function to fetch current volume data for each stock symbol
def fetch_current_volume(csv_file):
    try:
        if not os.path.exists(csv_file):
            print(f"CSV file not found: {csv_file}")
            return []

        # Read the CSV file containing stock symbols
        symbols_df = pd.read_csv(csv_file)
        if 'Symbol' not in symbols_df.columns:
            print(f"'Symbol' column not found in CSV file.")
            return []

        symbols_list = symbols_df['Symbol'].tolist()

        volume_data = []
        for symbol in symbols_list:
            try:
                stock = yf.Ticker(symbol)
                data = stock.history(period='1d')
                if not data.empty:
                    volume = data['Volume'].iloc[0]  # Use .iloc to avoid deprecation warning
                    volume_data.append((symbol, volume))
                else:
                    volume_data.append((symbol, 0))  # Append 0 if no data is available
            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")
                volume_data.append((symbol, 0))  # Append 0 in case of error

        # Create a DataFrame from the fetched volume data
        volume_df = pd.DataFrame(volume_data, columns=['Symbol', 'CurrentVolume'])

        # Sort the DataFrame by CurrentVolume in descending order and get the top 10 gainers
        top_10_volume_gainers = volume_df.sort_values(by='CurrentVolume', ascending=False).head(10)

        # Convert the top 10 gainers to the required format
        top_10_list = top_10_volume_gainers.to_dict(orient='records')
        return top_10_list

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Function to fetch top 10 price gainers
def fetch_top_price_gainers(csv_file):
    try:
        if not os.path.exists(csv_file):
            print(f"CSV file not found: {csv_file}")
            return []

        # Read the CSV file containing stock symbols
        symbols_df = pd.read_csv(csv_file)
        if 'Symbol' not in symbols_df.columns:
            print(f"'Symbol' column not found in CSV file.")
            return []

        symbols_list = symbols_df['Symbol'].tolist()

        price_data = []
        for symbol in symbols_list:
            try:
                stock = yf.Ticker(symbol)
                data = stock.history(period='1d')
                if not data.empty:
                    open_price = data['Open'].iloc[0]
                    close_price = data['Close'].iloc[0]
                    price_change = close_price - open_price
                    price_change_pct = (price_change / open_price) * 100
                    price_data.append((symbol, price_change, price_change_pct, close_price))
                else:
                    price_data.append((symbol, 0, 0, 0))  # Append 0 if no data is available
            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")
                price_data.append((symbol, 0, 0, 0))  # Append 0 in case of error

        # Create a DataFrame from the fetched price data
        price_df = pd.DataFrame(price_data, columns=['Symbol', 'PriceChange', 'PriceChangePct', 'CurrentPrice'])

        # Sort the DataFrame by PriceChange in descending order and get the top 10 gainers
        top_10_price_gainers = price_df.sort_values(by='PriceChange', ascending=False).head(10)

        # Convert the top 10 gainers to the required format
        top_10_list = top_10_price_gainers.to_dict(orient='records')
        return top_10_list

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Function to fetch top 10 price losers
def fetch_top_price_losers(csv_file):
    try:
        if not os.path.exists(csv_file):
            print(f"CSV file not found: {csv_file}")
            return []

        # Read the CSV file containing stock symbols
        symbols_df = pd.read_csv(csv_file)
        if 'Symbol' not in symbols_df.columns:
            print(f"'Symbol' column not found in CSV file.")
            return []

        symbols_list = symbols_df['Symbol'].tolist()

        price_data = []
        for symbol in symbols_list:
            try:
                stock = yf.Ticker(symbol)
                data = stock.history(period='1d')
                if not data.empty:
                    open_price = data['Open'].iloc[0]
                    close_price = data['Close'].iloc[0]
                    price_change = close_price - open_price
                    price_change_pct = (price_change / open_price) * 100
                    price_data.append((symbol, price_change, price_change_pct, close_price))
                else:
                    price_data.append((symbol, 0, 0, 0))  # Append 0 if no data is available
            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")
                price_data.append((symbol, 0, 0, 0))  # Append 0 in case of error

        # Create a DataFrame from the fetched price data
        price_df = pd.DataFrame(price_data, columns=['Symbol', 'PriceChange', 'PriceChangePct', 'CurrentPrice'])

        # Sort the DataFrame by PriceChange in ascending order and get the top 10 losers
        top_10_price_losers = price_df.sort_values(by='PriceChange', ascending=True).head(10)

        # Convert the top 10 losers to the required format
        top_10_list = top_10_price_losers.to_dict(orient='records')
        return top_10_list

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Step 1: Data Collection
def get_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

# Step 2: Data Preprocessing
def preprocess_data(data):
    data = data[['Close']]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

# Step 3: Model Building
def build_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 4: Model Training
def train_model(model, X_train, y_train, epochs=1, batch_size=1):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
    return model

# Step 5: Model Evaluation
def evaluate_model(model, X_test, y_test, scaler):
    predictions = model.predict(X_test)
    predictions = predictions.reshape(-1, 1)
    predictions = scaler.inverse_transform(predictions)
    y_test = y_test.reshape(-1, 1)
    y_test = scaler.inverse_transform(y_test)
    mse = mean_squared_error(y_test, predictions)
    accuracy = 100 - np.mean(np.abs((y_test - predictions) / y_test)) * 100
    return mse, accuracy, predictions

# Step 6: Multi-step Prediction
def predict_future_prices(model, last_100_days, scaler, future_steps):
    future_predictions = []
    current_input = last_100_days
    
    for _ in range(future_steps):
        current_input_scaled = scaler.transform(current_input)
        X_test = np.array([current_input_scaled])
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        pred_price = model.predict(X_test)
        pred_price_unscaled = scaler.inverse_transform(pred_price.reshape(-1, 1))
        future_predictions.append(pred_price_unscaled[0, 0])
        
        current_input = np.append(current_input, pred_price_unscaled)[-100:].reshape(-1, 1)
    
    return future_predictions


@app.route('/')
def home():
    return render_template('home1.html')

@app.route('/api/index_data/<index_name>', methods=['GET'])
def get_index_data(index_name):
    if index_name in index_data:
        data = index_data[index_name].reset_index().to_dict(orient='records')
        return jsonify(data)
    else:
        return jsonify({"error": "Index not found"}), 404



@app.route('/api/global_markets', methods=['GET'])
def get_global_markets():
    data = fetch_global_markets()
    return jsonify(data)

@app.route('/api/most_active', methods=['GET'])
def get_top_volume_gainers():
    csv_file = "C:/Users/adnan/Downloads/mw1.csv"  # Ensure this is the correct path to your CSV file
    data = fetch_current_volume(csv_file)
    return jsonify(data)

@app.route('/api/top_gainers', methods=['GET'])
def get_top_price_gainers():
    csv_file = "C:/Users/adnan/Downloads/mw1.csv"  # Ensure this is the correct path to your CSV file
    data = fetch_top_price_gainers(csv_file)
    return jsonify(data)

@app.route('/api/top_losers', methods=['GET'])
def get_top_price_losers():
    csv_file = "C:/Users/adnan/Downloads/mw1.csv"  # Ensure this is the correct path to your CSV file
    data = fetch_top_price_losers(csv_file)
    return jsonify(data)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    ticker = data['ticker']
    start_date = "2010-01-01"
    end_date = "2024-06-23"

    # Collect data
    stock_data = get_stock_data(ticker, start_date, end_date)

    # Preprocess data
    scaled_data, scaler = preprocess_data(stock_data)

    # Prepare training and test datasets
    training_size = int(len(scaled_data) * 0.8)
    train_data = scaled_data[:training_size, :]
    test_data = scaled_data[training_size:, :]

    time_step = 100
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)

    # Reshape inputs to be [samples, time steps, features]
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # Build model
    model = build_model((time_step, 1))

    # Train model
    model = train_model(model, X_train, y_train, epochs=10, batch_size=64)

    # Evaluate model
    mse, accuracy, predictions = evaluate_model(model, X_test, y_test, scaler)

    # Predict future prices
    last_100_days = stock_data['Close'][-100:].values.reshape(-1, 1)
    
    pred_tomorrow = predict_future_prices(model, last_100_days, scaler, 1)
    pred_next_week = predict_future_prices(model, last_100_days, scaler, 7)
    pred_next_month = predict_future_prices(model, last_100_days, scaler, 30)
    pred_next_six_months = predict_future_prices(model, last_100_days, scaler, 182)
    pred_next_year = predict_future_prices(model, last_100_days, scaler, 365)

    # Get the dates for plotting
    dates = stock_data.index[-len(y_test):].to_list()
    actual_prices = stock_data['Close'].values[-len(y_test):]
    
    future_dates_tomorrow = pd.date_range(start=dates[-1], periods=2, freq='B')[1:]
    future_dates_week = pd.date_range(start=dates[-1], periods=8, freq='B')[1:]
    future_dates_month = pd.date_range(start=dates[-1], periods=31, freq='B')[1:]
    future_dates_six_months = pd.date_range(start=dates[-1], periods=183, freq='B')[1:]
    future_dates_year = pd.date_range(start=dates[-1], periods=366, freq='B')[1:]

    return jsonify({
        'mse': round(float(mse), 2),
        'accuracy': round(float(accuracy), 2),
        'predicted_tomorrow': round(float(pred_tomorrow[0]), 2),
        'predicted_next_week': {
            'max': round(float(np.max(pred_next_week)), 2),
            'min': round(float(np.min(pred_next_week)), 2)
        },
        'predicted_next_month': {
            'max': round(float(np.max(pred_next_month)), 2),
            'min': round(float(np.min(pred_next_month)), 2)
        },
        'predicted_next_six_months': {
            'max': round(float(np.max(pred_next_six_months)), 2),
            'min': round(float(np.min(pred_next_six_months)), 2)
        },
        'predicted_next_year': {
            'max': round(float(np.max(pred_next_year)), 2),
            'min': round(float(np.min(pred_next_year)), 2)
        },
        'actual': {
            'dates': dates,
            'prices': actual_prices.tolist()
        },
        'predicted': {
            'dates': dates,
            'prices': predictions[:, 0].tolist()
        },
        'future_dates_week': future_dates_week.strftime('%Y-%m-%d').to_list(),
        'pred_next_week': np.array(pred_next_week).tolist(),
        'future_dates_month': future_dates_month.strftime('%Y-%m-%d').to_list(),
        'pred_next_month': np.array(pred_next_month).tolist(),
        'future_dates_six_months': future_dates_six_months.strftime('%Y-%m-%d').to_list(),
        'pred_next_six_months': np.array(pred_next_six_months).tolist(),
        'future_dates_year': future_dates_year.strftime('%Y-%m-%d').to_list(),
        'pred_next_year': np.array(pred_next_year).tolist()
    })


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False,port=8057)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8057
Press CTRL+C to quit
127.0.0.1 - - [30/Jun/2024 18:40:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2024 18:40:35] "GET /api/index_data/Nifty%2050 HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2024 18:40:35] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Jun/2024 18:40:35] "GET /api/index_data/Nifty%2050 HTTP/1.1" 200 -
^SSEC: No data found, symbol may be delisted
127.0.0.1 - - [30/Jun/2024 18:40:39] "GET /api/global_markets HTTP/1.1" 200 -
[*********************100%%**********************]  1 of 1 completed
C:\Users\adnan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10


127.0.0.1 - - [30/Jun/2024 18:40:44] "GET /api/top_losers HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2024 18:40:45] "GET /api/top_gainers HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2024 18:40:45] "GET /api/most_active HTTP/1.1" 200 -


44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - loss: 0.0101
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - loss: 1.6861e-04
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - loss: 1.3104e-04
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - loss: 1.1049e-04
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - loss: 1.0976e-04
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - loss: 9.6543e-05
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 1.2050e-04
Epoch 8/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - loss: 1.1538e-04
Epoch 9/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - loss: 9.8905e-05
Epoch 10/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - loss: 9.7408e-05
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


...

C:\Users\adnan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\adnan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\adnan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


C:\Users\adnan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\adnan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\adnan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


127.0.0.1 - - [30/Jun/2024 18:42:04] "POST /predict HTTP/1.1" 200 -
